In [6]:
subject_id = "100307"
data_path = "/headnode1/abry4213/data/HCP100/"
SPI_subset = "/headnode1/abry4213/github/Directed_information_fMRI/feature_extraction/pyspi/fast_extended.yaml"
ROI_lookup_file = "/headnode1/abry4213/github/Directed_information_fMRI/Brain_Region_info.csv"

In [23]:
brain_region_lookup = pd.read_csv("../../Brain_Region_info.csv", index_col=False).reset_index(drop=True)
base_regions = list(set(brain_region_lookup.Base_Region.tolist()))

In [24]:
brain_region_lookup

,Region_Index,Brain_Region,ggseg,Hemisphere,Base_Region
0,0,ctx-lh-bankssts,ctx-lh-bankssts,Left,bankssts
1,1,ctx-lh-caudalanteriorcingulate,ctx-lh-caudalanteriorcingulate,Left,caudalanteriorcingulate
2,2,ctx-lh-caudalmiddlefrontal,ctx-lh-caudalmiddlefrontal,Left,caudalmiddlefrontal
3,3,ctx-lh-cuneus,ctx-lh-cuneus,Left,cuneus
4,4,ctx-lh-entorhinal,ctx-lh-entorhinal,Left,entorhinal
...,...,...,...,...,...
63,63,ctx-rh-supramarginal,ctx-rh-supramarginal,Right,supramarginal
64,64,ctx-rh-frontalpole,ctx-rh-frontalpole,Right,frontalpole
65,65,ctx-rh-temporalpole,ctx-rh-temporalpole,Right,temporalpole
66,66,ctx-rh-transversetemporal,ctx-rh-transversetemporal,Right,transversetemporal


In [20]:
calc_file = "/headnode1/abry4213/data/HCP100/time_series_features/100307/calc.pkl"

# Load calc_file, which is a .pkl file
with open(calc_file, 'rb') as f:
    homotopic_FC_res = (pickle.load(f)
     .query("brain_region_from != brain_region_to")
     .rename(columns={"brain_region_from": "region_from_index", "brain_region_to": "region_to_index"})
     .reset_index(drop=True)
     .assign(region_from_index = lambda x: x.region_from_index.str.replace("proc-", ""),
            region_to_index = lambda x: x.region_to_index.str.replace("proc-", ""))
     .astype({"region_from_index": int, "region_to_index": int})
     .merge(brain_region_lookup[["Region_Index", "Base_Region", "Hemisphere"]], left_on="region_from_index", right_on="Region_Index", how="left")
     .rename(columns={"Base_Region": "base_region_from", "Hemisphere": "hemi_from"})
     .drop(columns=["Region_Index"])
     .merge(brain_region_lookup[["Region_Index", "Base_Region", "Hemisphere"]], left_on="region_to_index", right_on="Region_Index", how="left")
     .rename(columns={"Base_Region": "base_region_to", "Hemisphere": "hemi_to"})
     .drop(columns=["Region_Index", "region_from_index", "region_to_index"])
     .query("base_region_from == base_region_to & hemi_from != hemi_to")
    )
homotopic_FC_res.head()

,value,SPI,base_region_from,hemi_from,base_region_to,hemi_to
33,0.378645,cov_EmpiricalCovariance,bankssts,Right,bankssts,Left
101,0.183121,cov_EmpiricalCovariance,caudalanteriorcingulate,Right,caudalanteriorcingulate,Left
169,0.347736,cov_EmpiricalCovariance,caudalmiddlefrontal,Right,caudalmiddlefrontal,Left
237,0.784956,cov_EmpiricalCovariance,cuneus,Right,cuneus,Left
305,0.057536,cov_EmpiricalCovariance,entorhinal,Right,entorhinal,Left


In [7]:

# Get the base name for SPI_subset file
SPI_subset_base = op.basename(SPI_subset).replace(".yaml", "")

# Time series output path for this subject
time_series_path = f"{data_path}/raw_data/functional_MRI/numpy_files/"
output_feature_path = op.join(data_path, "time_series_features/pyspi")

# Define ROI lookup table
ROI_lookup = pd.read_csv(ROI_lookup_file)
region_names = list(ROI_lookup.Brain_Region.values)

# Check if pyspi results already exist for this subject
output_file = f"{output_feature_path}/HCP100_{subject_id}_all_pyspi_{SPI_subset_base}_results.csv"
if op.isfile(output_file):
    print(f"{SPI_subset_base} SPI results for {subject_id} already exist. Skipping.")
    exit() 

# Load in the .npy file
npy_file = f"{time_series_path}/{subject_id}.npy"
TS_data = np.load(npy_file)

In [10]:
# calc = Calculator(configfile=SPI_subset)
calc = Calculator(subset='fabfour')

# Make deepcopy of calc 
calc_copy = deepcopy(calc)

# Load data 
calc_copy.load_dataset(TS_data[:2,:])
calc_copy.compute()


Loading configuration file: /headnode1/abry4213/.conda/envs/annie_env/lib/python3.9/site-packages/pyspi/fabfour_config.yaml
*** Importing module .statistics.basic
[0] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'EmpiricalCovariance'})
Succesfully initialised SPI with identifier "cov_EmpiricalCovariance" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[1] Adding SPI .statistics.basic.SpearmanR(x,y,{'squared': True})
Succesfully initialised SPI with identifier "spearmanr-sq" and labels ['basic', 'unordered', 'rank', 'linear', 'undirected', 'unsigned', 'unsigned']
*** Importing module .statistics.infotheory
[2] Adding SPI .statistics.infotheory.DirectedInfo(x,y,{'estimator': 'gaussian'})
Succesfully initialised SPI with identifier "di_gaussian" and labels ['unsigned', 'infotheory', 'temporal', 'directed', 'linear']
*** Importing module .statistics.misc
[3] Adding SPI .statistics.misc.PowerEnvelopeCorrelation(x,y,{'orth': False, 'log': False, 'absolute': F

Processing [None: pec]: 100%|██████████| 4/4 [00:00<00:00,  8.13it/s]            


Calculation complete. Time taken: 0.4926s

SPI Computation Results Summary

Total number of SPIs attempted: 4
Number of SPIs successfully computed: 4 (100.00%)
------------------------------------------------------------
Category       | Count | Percentage
------------------------------------------------------------
Successful     |     4 | 100.00%
NaNs           |     0 |   0.00%
Partial NaNs   |     0 |   0.00%
------------------------------------------------------------


In [16]:
# Iterate over each SPI
SPI_res = deepcopy(calc_copy.table)
SPI_res.columns = SPI_res.columns.to_flat_index()

SPI_res = SPI_res.rename(columns='__'.join).assign(region_from = lambda x: x.index)
SPI_res_long = SPI_res.melt(id_vars='region_from', var_name='SPI__region_to', value_name='value')
SPI_res_long["SPI"] = SPI_res_long["SPI__region_to"].str.split("__").str[0]
SPI_res_long["region_to"] = SPI_res_long["SPI__region_to"].str.split("__").str[1]

In [17]:
SPI_res_long.head()

,region_from,SPI__region_to,value,SPI,region_to
0,proc-0,cov_EmpiricalCovariance__proc-0,NaN,cov_EmpiricalCovariance,proc-0
1,proc-1,cov_EmpiricalCovariance__proc-0,-0.09525,cov_EmpiricalCovariance,proc-0
2,proc-0,cov_EmpiricalCovariance__proc-1,-0.09525,cov_EmpiricalCovariance,proc-1
3,proc-1,cov_EmpiricalCovariance__proc-1,NaN,cov_EmpiricalCovariance,proc-1
4,proc-0,spearmanr-sq__proc-0,NaN,spearmanr-sq,proc-0


In [34]:
(SPI_res_long.rename(columns={"region_from": "region_from_index", "region_to": "region_to_index"})
    .reset_index(drop=True)
    .assign(region_from_index = lambda x: x.region_from_index.str.replace("proc-", ""),
        region_to_index = lambda x: x.region_to_index.str.replace("proc-", ""))
    .astype({"region_from_index": int, "region_to_index": int})
    .merge(brain_region_lookup[["Region_Index", "Base_Region", "Hemisphere"]], left_on="region_from_index", right_on="Region_Index", how="left")
    .rename(columns={"Base_Region": "base_region_from", "Hemisphere": "hemi_from"})
    .drop(columns=["Region_Index"])
    .merge(brain_region_lookup[["Region_Index", "Base_Region", "Hemisphere"]], left_on="region_to_index", right_on="Region_Index", how="left")
    .rename(columns={"Base_Region": "base_region_to", "Hemisphere": "hemi_to"})
    .drop(columns=["Region_Index", "region_from_index", "region_to_index", "SPI__region_to"])
    # .query("base_region_from == base_region_to & hemi_from != hemi_to")
)

,value,SPI,base_region_from,hemi_from,base_region_to,hemi_to
0,NaN,cov_EmpiricalCovariance,bankssts,Left,bankssts,Left
1,-0.095250,cov_EmpiricalCovariance,caudalanteriorcingulate,Left,bankssts,Left
2,-0.095250,cov_EmpiricalCovariance,bankssts,Left,caudalanteriorcingulate,Left
3,NaN,cov_EmpiricalCovariance,caudalanteriorcingulate,Left,caudalanteriorcingulate,Left
4,NaN,spearmanr-sq,bankssts,Left,bankssts,Left
5,0.009611,spearmanr-sq,caudalanteriorcingulate,Left,bankssts,Left
6,0.009611,spearmanr-sq,bankssts,Left,caudalanteriorcingulate,Left
7,NaN,spearmanr-sq,caudalanteriorcingulate,Left,caudalanteriorcingulate,Left
8,NaN,di_gaussian,bankssts,Left,bankssts,Left
9,0.192620,di_gaussian,caudalanteriorcingulate,Left,bankssts,Left
